# Testing Evolutionary algorithms with DEAP

**S. Doncieux, Sorbonne University, 2022, [stephane.doncieux@sorbonne-universite.fr](mailto:stephane.doncieux@sorbonne-universite.fr)**

_UE Robotique et Apprentissage, Master ANDROIDE_

## Introduction to DEAP

Mostly extracted from https://deap.readthedocs.io/en/master/overview.html

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import numpy as np
import importlib
import random

random.seed()

from deap import base, creator, benchmarks

from deap import tools


## A simple example

In [2]:
## We create the types we need ##

# Fitness that is minimized (-1 weight)
if (hasattr(creator, "FitnessMin")):
    # Deleting any previous definition (to avoid warning message)
    del creator.FitnessMin
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Individual that uses this fitness
if (hasattr(creator, "Individual")):
    # Deleting any previous definition (to avoid warning message)
    del creator.Individual
creator.create("Individual", list, fitness=creator.FitnessMin)

## Tool initialization ##
IND_SIZE = 5

# toolbox is a container, each registered function can be called later on. Example:
# toolbox.register("my_function", my_great_function, default_param=42)
# toobox.my_function(...) calls my_great_function(...)
# some parameters with default values can be defined when registering the function, 
# they are then transmitted to it when it is called 
# (in the example, the param default_param is transmitted to the function with the value 42)
toolbox = base.Toolbox()

# parameters are initialized between 0 and 1
toolbox.register("attribute", random.random)

# individuals are made with IND_SIZE parameters
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)

# the population is a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    return sum(individual),

# 2 points crossover
toolbox.register("mate", tools.cxTwoPoint)

# gaussian mutation
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)

# Tournament selection: select the best out of X randomly selected individuals (with X=3)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evaluation function to use
toolbox.register("evaluate", evaluate)


In [3]:
def main(NGEN=40, POPSIZE=10):

    # Initialising the population
    pop = toolbox.population(n=POPSIZE)
    CXPB, MUTPB = 0.5, 0.2

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

In [4]:
pop=main(100)

In [5]:
for i,ind in enumerate(pop):
    print("Indiv[%d]=%f"%(i,evaluate(ind)[0])+" "+str(ind))

Indiv[0]=-28.809522 [-5.760450973119745, -5.376401806184157, -5.865942791830913, -3.249416667206033, -8.557309571352523]
Indiv[1]=-28.979408 [-5.760450973119745, -5.376401806184157, -6.03582925414387, -3.249416667206033, -8.557309571352523]
Indiv[2]=-28.809522 [-5.760450973119745, -5.376401806184157, -5.865942791830913, -3.249416667206033, -8.557309571352523]
Indiv[3]=-28.963300 [-5.760450973119745, -5.376401806184157, -6.03582925414387, -3.2333085204969767, -8.557309571352523]
Indiv[4]=-28.979408 [-5.760450973119745, -5.376401806184157, -6.03582925414387, -3.249416667206033, -8.557309571352523]
Indiv[5]=-28.793414 [-5.760450973119745, -5.376401806184157, -5.865942791830913, -3.2333085204969767, -8.557309571352523]
Indiv[6]=-28.793414 [-5.760450973119745, -5.376401806184157, -5.865942791830913, -3.2333085204969767, -8.557309571352523]
Indiv[7]=-28.979408 [-5.760450973119745, -5.376401806184157, -6.03582925414387, -3.249416667206033, -8.557309571352523]
Indiv[8]=-28.963300 [-5.760450973